In [96]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
import numpy as np
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from PIL import Image, ImageFile
from random import random
from os.path import join, dirname
from data.dataset_utils import *
import pandas as pd
import os
import warnings
import torchvision.models as models

warnings.filterwarnings('ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Image.LOAD_TRUNCATED_IMAGES = True
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [97]:
# returns kaggle
def get_kaggle_retinopathy_dataset_info(req_set = 'train'):

    # kaggle_csv_file_path = '/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/trainLabels.csv'
    # truncated one
    kaggle_csv_file_path = '/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/truncated_trainLabels.csv'
    kaggle_image_path = '/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/train/'

    data_file = pd.read_csv(kaggle_csv_file_path, header=0)
    # print(data_file.head())
    db_len = 0
    start = 0
    end = 0
    # 0.7 train, 0.15 val, 0.15 test
    if req_set == 'train':
        start = 0
        # end = 24588
        end = int(0.7 * len(data_file))
    elif req_set == 'val':
        start = int(0.7 * len(data_file))
        # end = 29857
#         end = int(0.85 * len(data_file))
        end = len(data_file) - 1
    elif req_set == 'test':
        start = int(0.85 * len(data_file))
        end = len(data_file) - 1
        
    elif req_set == 'full':
        start = 0
        end = len(data_file) - 1

    name_list = []
    label_list = []

    for i in range(start, end):
        file_name = data_file.iloc[i][0] + '.jpeg'
        if os.path.isfile(os.path.join(kaggle_image_path, file_name)):
            name_list.append(os.path.join(kaggle_image_path, file_name))
            label_list.append(data_file.iloc[i][1])
        else:
            print()

    # for idx, row in data_file.iterrows():
    #     if os.path.isfile(os.path.join(fgadr_image_path, row[0])):
    #         name_list.append(os.path.join(fgadr_image_path, row[0]))
    #         label_list.append(row[1])
    #     else:
    #         print()

    return name_list, label_list


def get_retinopathy_dataset_info(req_set = 'train'):
    fgadr_root_path = '/mnt/sda/haal02-data/FGADR-Seg-Set/Seg-set'
    fgadr_csv_file_name = 'DR_Seg_Grading_Label.csv'
    fgadr_image_path = '/mnt/sda/haal02-data/FGADR-Seg-Set/Seg-set/Original_Images'

    data_file = pd.read_csv(os.path.join(fgadr_root_path, fgadr_csv_file_name), header=None)

    db_len = 0
    start = 0
    end = 0
    # 0.7 train, 0.15 val, 0.15 test
    if req_set == 'train':
        start = 0
        end = 1450
    elif req_set == 'val':
        start = 1450
#         end = 1565
        end = len(data_file) - 1
    elif req_set == 'test':
        start = 1565
        end = len(data_file) - 1
        
    elif req_set == 'full':
        start = 0
        end = len(data_file) - 1

    name_list = []
    label_list = []

    for i in range(start, end):
        if os.path.isfile(os.path.join(fgadr_image_path, data_file.iloc[i][0])):
            name_list.append(os.path.join(fgadr_image_path, data_file.iloc[i][0]))
            label_list.append(data_file.iloc[i][1])
        else:
            print()

    # for idx, row in data_file.iterrows():
    #     if os.path.isfile(os.path.join(fgadr_image_path, row[0])):
    #         name_list.append(os.path.join(fgadr_image_path, row[0]))
    #         label_list.append(row[1])
    #     else:
    #         print()

    return name_list, label_list



In [98]:
# Dataset Class

class FGADRDataset(data.Dataset):
    def __init__(self, name, split='train', val_size=0, rot_classes=3,
            img_transformer=None, bias_whole_image=None, mode='RGB'):
        # if split == 'train':
        #     names, _, labels, _ = get_split_dataset_info(join(dirname(__file__), 'txt_lists', '%s_train.txt' % name), val_size)
        # elif split =='val':
        #     _, names, _, labels = get_split_dataset_info(join(dirname(__file__), 'txt_lists', '%s_train.txt' % name), val_size)
        # elif split == 'test':
        #     names, labels = get_dataset_info(join(dirname(__file__), 'txt_lists', '%s_test.txt' % name))

        print("Dataset Name: ", name)
        if name == 'FGADR':
            names, labels = get_retinopathy_dataset_info(req_set=split)
        elif name == 'KaggleR':
            names, labels = get_kaggle_retinopathy_dataset_info(req_set=split)


        # print(names, labels)
        # self.data_path = join(dirname(__file__), '..', 'datasets')
        self.data_path = '/mnt/sda/haal02-data/FGADR-Seg-Set/Seg-set/Original_Images'
        self.names = names
        self.labels = labels
        
        self.rot_classes = rot_classes
        self.mode = mode

        self.N = len(self.names)
        self.bias_whole_image = bias_whole_image
        self._image_transformer = img_transformer
        
        
    def get_image(self, index):
        # framename = self.data_path + '/' + self.names[index]
        framename = self.names[index]
        img = Image.open(framename).convert(self.mode)
        img = img.resize((512, 512), Image.NEAREST)
        return img

    
    def __getitem__(self, index):
        
        img = self.get_image(index)

        img = self._image_transformer(img)
        target = self.labels[index]
        
        return img, target
    
#         sample = {'images': data,
#                 'images_ori': self._image_transformer(rot_imgs[0]),
#                 'aux_labels': int(order),
#                 'class_labels': int(self.labels[index])}
#         return sample

    def __len__(self):
        return len(self.names)



In [99]:
batch_size = 64
num_epochs = 100
learning_rate = 0.01
learning_rate_decay = 0.95

# FGADR Dataloaders

In [100]:
fgadr_norm_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean = [0.4557, 0.2588, 0.1325],
                                                        std = [0.2856, 0.1855, 0.1356])
                                     ])


f_train_set = FGADRDataset(name='FGADR', split='train', img_transformer=fgadr_norm_transform)
f_val_set = FGADRDataset(name='FGADR', split='val', img_transformer=fgadr_norm_transform)
f_test_set = FGADRDataset(name='FGADR', split='test', img_transformer=fgadr_norm_transform)

f_train_loader = torch.utils.data.DataLoader(f_train_set, batch_size=batch_size, shuffle=True, num_workers=4)
f_val_loader = torch.utils.data.DataLoader(f_val_set, batch_size=batch_size, shuffle=False)
f_test_loader = torch.utils.data.DataLoader(f_test_set, batch_size=batch_size, shuffle=False)

Dataset Name:  FGADR
Dataset Name:  FGADR
Dataset Name:  FGADR


In [101]:
basic_transform = transforms.Compose([transforms.ToTensor(),
                                     ])

f_full_set = FGADRDataset(name='FGADR', split='full', img_transformer=basic_transform)
f_full_loader = torch.utils.data.DataLoader(f_full_set, batch_size=batch_size, shuffle=False, num_workers=4)
print(len(f_full_set))

Dataset Name:  FGADR
1841


# KaggleR Dataset

In [102]:
#Kaggle set
kaggleR_norm_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean = [0.3150, 0.2218, 0.1578],
                                                        std = [0.3002, 0.2178, 0.1720])
                                     ])
k_train_set = FGADRDataset(name='KaggleR', split='train', img_transformer=kaggleR_norm_transform)
k_val_set = FGADRDataset(name='KaggleR', split='val', img_transformer=kaggleR_norm_transform)
k_test_set = FGADRDataset(name='KaggleR', split='test', img_transformer=kaggleR_norm_transform)

k_train_loader = torch.utils.data.DataLoader(k_train_set, batch_size=batch_size, shuffle=False, num_workers=4)

k_val_loader = torch.utils.data.DataLoader(k_val_set, batch_size=batch_size, shuffle=False)
k_test_loader = torch.utils.data.DataLoader(k_test_set, batch_size=batch_size, shuffle=False)
print(len(k_train_set), len(k_val_set))

Dataset Name:  KaggleR
Dataset Name:  KaggleR
Dataset Name:  KaggleR
10721 4594


In [103]:
basic_transform = transforms.Compose([transforms.ToTensor(),
                                     ])

k_full_set = FGADRDataset(name='KaggleR', split='full', img_transformer=basic_transform)
k_full_loader = torch.utils.data.DataLoader(k_full_set, batch_size=128, shuffle=False, num_workers=4)
print(len(k_full_set))

Dataset Name:  KaggleR
15315


In [89]:
# # computing mean and variance
# mean = 0.0
# for datas in fgadr_full_loader:
#     images, labels = datas
# #     images = datas['images']
#     batch_samples = images.size(0)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
    
# mean = mean / len(fgadr_full_loader.dataset)

# var = 0.0
# for datas in fgadr_full_loader:
#     images, labels = datas
#     batch_samples = images.size(0)
# #     images = data['images']
#     images = images.view(batch_samples, images.size(1), -1)
#     var += ((images - mean.unsqueeze(1)) ** 2).sum([0, 2])

# std = torch.sqrt(var / (len(fgadr_full_loader.dataset) * 224 * 224))

# print(len(fgadr_full_loader.dataset))
# print("Mean and std: ", mean, std)

In [90]:
# for datas in train_loader:
#     img, label = datas
#     print(img.shape)

In [104]:
learning_rate = 0.01
num_epochs = 150
def train_function(model, train_loader='', val_loader='', test_loader='', train=True, save_model_path='model.ckpt'):
    
    if train:
#         model.apply(weights_init)


        # Loss and optimizer
        c_weight = torch.FloatTensor([0.235, 0.2225, 0.17, 0.16, 0.21250001]).to(device)
        criterion = nn.CrossEntropyLoss(weight = c_weight)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#         optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                             step_size=50, gamma=0.1)

        # Train the model
        lr = learning_rate
        total_step = len(train_loader)
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(train_loader):
                # Move tensors to the configured device
                images = images.to(device)
                labels = labels.to(device)



                optimizer.zero_grad()
#                 outputs = model(images.view(batch_size, -1))
                outputs = model(images)
                loss = criterion(outputs, labels.long())

                loss.backward()
                optimizer.step()
                

                if (i+1) % 10 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

            # Code to update the lr
#             lr *= learning_rate_decay
#             update_lr(optimizer, lr)
            exp_lr_scheduler.step()
            
            if epoch%2 == 0:
                with torch.no_grad():
                    correct = 0
                    total = 0
                    for images, labels in val_loader:
                        images = images.to(device)
                        labels = labels.to(device)



                        predicted = model(images)
                        _, predicted = torch.max(predicted, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()


                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                    print(' Lr: {} , Validataion accuracy is: {} %'.format(optimizer.param_groups[0]['lr'],(100 * correct / total)))


                # Save the model checkpoint
                torch.save(model.state_dict(), save_model_path)

    else:
        # Run the test code once you have your by setting train flag to false
        # and loading the best model

        best_model = torch.load(save_model_path)
        model.load_state_dict(best_model)
        # Test the model
        # In test phase, we don't need to compute gradients (for memory efficiency)
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)


                predicted = model(images)
                _, predicted = torch.max(predicted, 1)


                # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
#                 if total == 1000:
#                     break

            print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))



# Training on FGADR

In [105]:
model_path_to_save = 'fgadr_trained_model.ckpt'
resnet18 = models.resnet18(pretrained=True)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, 5)
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [106]:
train_function(resnet18, train_loader=f_train_loader, val_loader=f_val_loader, test_loader='', train=True, save_model_path=model_path_to_save)

Epoch [1/150], Step [10/23], Loss: 2.1629
Epoch [1/150], Step [20/23], Loss: 1.3715
 Lr: 0.01 , Validataion accuracy is: 50.12787723785166 %
Epoch [2/150], Step [10/23], Loss: 1.3581
Epoch [2/150], Step [20/23], Loss: 1.3760
Epoch [3/150], Step [10/23], Loss: 1.1503
Epoch [3/150], Step [20/23], Loss: 1.1549
 Lr: 0.01 , Validataion accuracy is: 57.289002557544755 %
Epoch [4/150], Step [10/23], Loss: 1.3042
Epoch [4/150], Step [20/23], Loss: 1.1543
Epoch [5/150], Step [10/23], Loss: 1.2099
Epoch [5/150], Step [20/23], Loss: 1.1481
 Lr: 0.01 , Validataion accuracy is: 57.289002557544755 %
Epoch [6/150], Step [10/23], Loss: 1.0719
Epoch [6/150], Step [20/23], Loss: 1.2979
Epoch [7/150], Step [10/23], Loss: 1.0051
Epoch [7/150], Step [20/23], Loss: 0.9778
 Lr: 0.01 , Validataion accuracy is: 56.010230179028135 %
Epoch [8/150], Step [10/23], Loss: 1.0685
Epoch [8/150], Step [20/23], Loss: 1.1016
Epoch [9/150], Step [10/23], Loss: 0.9890
Epoch [9/150], Step [20/23], Loss: 1.0472
 Lr: 0.01 , V

Epoch [72/150], Step [20/23], Loss: 0.5811
Epoch [73/150], Step [10/23], Loss: 0.7261
Epoch [73/150], Step [20/23], Loss: 0.7159
 Lr: 0.001 , Validataion accuracy is: 60.10230179028133 %
Epoch [74/150], Step [10/23], Loss: 0.9395
Epoch [74/150], Step [20/23], Loss: 0.7191
Epoch [75/150], Step [10/23], Loss: 0.6858
Epoch [75/150], Step [20/23], Loss: 0.6199
 Lr: 0.001 , Validataion accuracy is: 60.61381074168798 %
Epoch [76/150], Step [10/23], Loss: 0.7608
Epoch [76/150], Step [20/23], Loss: 0.8014
Epoch [77/150], Step [10/23], Loss: 0.6667
Epoch [77/150], Step [20/23], Loss: 0.6201
 Lr: 0.001 , Validataion accuracy is: 57.289002557544755 %
Epoch [78/150], Step [10/23], Loss: 0.7694
Epoch [78/150], Step [20/23], Loss: 0.7811
Epoch [79/150], Step [10/23], Loss: 0.6808
Epoch [79/150], Step [20/23], Loss: 0.7646
 Lr: 0.001 , Validataion accuracy is: 58.05626598465473 %
Epoch [80/150], Step [10/23], Loss: 0.6904
Epoch [80/150], Step [20/23], Loss: 0.7187
Epoch [81/150], Step [10/23], Loss: 

Epoch [143/150], Step [10/23], Loss: 0.3175
Epoch [143/150], Step [20/23], Loss: 0.2339
 Lr: 0.0001 , Validataion accuracy is: 59.846547314578004 %
Epoch [144/150], Step [10/23], Loss: 0.2256
Epoch [144/150], Step [20/23], Loss: 0.2854
Epoch [145/150], Step [10/23], Loss: 0.2046
Epoch [145/150], Step [20/23], Loss: 0.1638
 Lr: 0.0001 , Validataion accuracy is: 60.61381074168798 %
Epoch [146/150], Step [10/23], Loss: 0.1460
Epoch [146/150], Step [20/23], Loss: 0.2296
Epoch [147/150], Step [10/23], Loss: 0.1361
Epoch [147/150], Step [20/23], Loss: 0.2083
 Lr: 0.0001 , Validataion accuracy is: 61.12531969309463 %
Epoch [148/150], Step [10/23], Loss: 0.1515
Epoch [148/150], Step [20/23], Loss: 0.1413
Epoch [149/150], Step [10/23], Loss: 0.2162
Epoch [149/150], Step [20/23], Loss: 0.2822
 Lr: 0.0001 , Validataion accuracy is: 58.05626598465473 %
Epoch [150/150], Step [10/23], Loss: 0.1691
Epoch [150/150], Step [20/23], Loss: 0.2680


In [63]:
cl_0 = 0
cl_1 = 0
cl_2 = 0
cl_3 = 0
cl_4 = 0
for data in f_train_loader:
    img, label = data
    for lbl in label:
        if lbl == 0:
            cl_0 += 1
        elif lbl == 1:
            cl_1 += 1
        elif lbl == 2:
            cl_2 += 1
        elif lbl == 3:
            cl_3 += 1
        elif lbl == 4:
            cl_4 += 1
            
print(cl_0, cl_1, cl_2, cl_3, cl_4)

84 164 464 521 217


In [72]:
val_list = [84, 164, 464, 521, 217]
weight_tensor = torch.zeros(1,5)
for idx in range(len(val_list)):
    wt = val_list[idx] / len(f_train_loader.dataset)
#     print(len(data_file))
    weight_tensor[0][idx] = round((1 - wt), 2)
    
from sklearn.preprocessing import normalize
normalize(weight_tensor, norm='l1')

array([[0.235     , 0.2225    , 0.17      , 0.16      , 0.21250001]])

In [64]:
cl_0 = 0
cl_1 = 0
cl_2 = 0
cl_3 = 0
cl_4 = 0
for data in f_val_loader:
    img, label = data
    for lbl in label:
        if lbl == 0:
            cl_0 += 1
        elif lbl == 1:
            cl_1 += 1
        elif lbl == 2:
            cl_2 += 1
        elif lbl == 3:
            cl_3 += 1
        elif lbl == 4:
            cl_4 += 1
            
print(cl_0, cl_1, cl_2, cl_3, cl_4)

17 48 131 125 70


In [30]:
train_function(resnet18, test_loader=k_full_loader, train=False, save_model_path=model_path_to_save)

Accuracy of the network on the 15315 test images: 16.643813254978777 %


Dataset Name:  KaggleR
Dataset Name:  KaggleR
Dataset Name:  KaggleR


In [114]:
len(train_set)

10721

In [83]:
# train_function(resnet18, test_loader=k_test_loader, train=False)

Accuracy of the network on the 2297 test images: 27.514148889856333 %


In [86]:
# train_function(resnet18, test_loader=k_train_loader, train=False)

Accuracy of the network on the 10721 test images: 14.718776233560302 %


# Kaggle model Training

In [107]:
k_model_path_to_save = 'kaggle_trained_model.ckpt'
k_resnet18 = models.resnet18(pretrained=True)
num_features = k_resnet18.fc.in_features
k_resnet18.fc = nn.Linear(num_features, 5)
k_resnet18.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
train_function(k_resnet18, train_loader=k_train_loader, val_loader=k_val_loader, train=True, save_model_path=k_model_path_to_save)


Epoch [1/150], Step [10/168], Loss: 1.6579
Epoch [1/150], Step [20/168], Loss: 0.8643
Epoch [1/150], Step [30/168], Loss: 0.6707
Epoch [1/150], Step [40/168], Loss: 1.0035
Epoch [1/150], Step [50/168], Loss: 0.6731
Epoch [1/150], Step [60/168], Loss: 0.9568
Epoch [1/150], Step [70/168], Loss: 0.6245
Epoch [1/150], Step [80/168], Loss: 0.5826
Epoch [1/150], Step [90/168], Loss: 0.6032
Epoch [1/150], Step [100/168], Loss: 0.8267
Epoch [1/150], Step [110/168], Loss: 0.6488
Epoch [1/150], Step [120/168], Loss: 0.6168
Epoch [1/150], Step [130/168], Loss: 2.7461
Epoch [1/150], Step [140/168], Loss: 1.0346
Epoch [1/150], Step [150/168], Loss: 1.2063
Epoch [1/150], Step [160/168], Loss: 1.2759
 Lr: 0.01 , Validataion accuracy is: 56.24727905964301 %
Epoch [2/150], Step [10/168], Loss: 1.6273
Epoch [2/150], Step [20/168], Loss: 1.0229
Epoch [2/150], Step [30/168], Loss: 0.5721
Epoch [2/150], Step [40/168], Loss: 0.7239
Epoch [2/150], Step [50/168], Loss: 0.6368
Epoch [2/150], Step [60/168], Los

Epoch [12/150], Step [50/168], Loss: 0.7510
Epoch [12/150], Step [60/168], Loss: 0.8343
Epoch [12/150], Step [70/168], Loss: 0.7225
Epoch [12/150], Step [80/168], Loss: 0.6193
Epoch [12/150], Step [90/168], Loss: 0.5947
Epoch [12/150], Step [100/168], Loss: 0.8055
Epoch [12/150], Step [110/168], Loss: 0.6868
Epoch [12/150], Step [120/168], Loss: 0.6539
Epoch [12/150], Step [130/168], Loss: 2.2656
Epoch [12/150], Step [140/168], Loss: 2.1450
Epoch [12/150], Step [150/168], Loss: 2.0634
Epoch [12/150], Step [160/168], Loss: 1.9638
Epoch [13/150], Step [10/168], Loss: 0.9573
Epoch [13/150], Step [20/168], Loss: 0.9878
Epoch [13/150], Step [30/168], Loss: 0.8965
Epoch [13/150], Step [40/168], Loss: 0.8744
Epoch [13/150], Step [50/168], Loss: 0.7563
Epoch [13/150], Step [60/168], Loss: 0.8412
Epoch [13/150], Step [70/168], Loss: 0.7209
Epoch [13/150], Step [80/168], Loss: 0.6263
Epoch [13/150], Step [90/168], Loss: 0.5955
Epoch [13/150], Step [100/168], Loss: 0.8058
Epoch [13/150], Step [11

Epoch [23/150], Step [70/168], Loss: 0.6527
Epoch [23/150], Step [80/168], Loss: 0.5626
Epoch [23/150], Step [90/168], Loss: 0.4990
Epoch [23/150], Step [100/168], Loss: 0.7704
Epoch [23/150], Step [110/168], Loss: 0.6596
Epoch [23/150], Step [120/168], Loss: 0.6293
Epoch [23/150], Step [130/168], Loss: 2.3106
Epoch [23/150], Step [140/168], Loss: 1.6323
Epoch [23/150], Step [150/168], Loss: 1.3059
Epoch [23/150], Step [160/168], Loss: 1.2493
 Lr: 0.01 , Validataion accuracy is: 56.24727905964301 %
Epoch [24/150], Step [10/168], Loss: 1.5933
Epoch [24/150], Step [20/168], Loss: 0.8830
Epoch [24/150], Step [30/168], Loss: 0.5718
Epoch [24/150], Step [40/168], Loss: 0.7278
Epoch [24/150], Step [50/168], Loss: 0.6224
Epoch [24/150], Step [60/168], Loss: 0.8100
Epoch [24/150], Step [70/168], Loss: 0.7037
Epoch [24/150], Step [80/168], Loss: 0.5480
Epoch [24/150], Step [90/168], Loss: 0.4894
Epoch [24/150], Step [100/168], Loss: 0.7779
Epoch [24/150], Step [110/168], Loss: 0.6511
Epoch [24/

Epoch [34/150], Step [80/168], Loss: 0.5223
Epoch [34/150], Step [90/168], Loss: 0.5048
Epoch [34/150], Step [100/168], Loss: 0.7763
Epoch [34/150], Step [110/168], Loss: 0.6589
Epoch [34/150], Step [120/168], Loss: 0.6229
Epoch [34/150], Step [130/168], Loss: 2.2687
Epoch [34/150], Step [140/168], Loss: 1.5409
Epoch [34/150], Step [150/168], Loss: 1.3226
Epoch [34/150], Step [160/168], Loss: 1.2656
Epoch [35/150], Step [10/168], Loss: 2.2739
Epoch [35/150], Step [20/168], Loss: 0.9697
Epoch [35/150], Step [30/168], Loss: 0.5903
Epoch [35/150], Step [40/168], Loss: 0.7141
Epoch [35/150], Step [50/168], Loss: 0.6374
Epoch [35/150], Step [60/168], Loss: 0.7821
Epoch [35/150], Step [70/168], Loss: 0.6768
Epoch [35/150], Step [80/168], Loss: 0.5393
Epoch [35/150], Step [90/168], Loss: 0.5175
Epoch [35/150], Step [100/168], Loss: 0.7712
Epoch [35/150], Step [110/168], Loss: 0.6673
Epoch [35/150], Step [120/168], Loss: 0.6204
Epoch [35/150], Step [130/168], Loss: 2.2396
Epoch [35/150], Step 

Epoch [45/150], Step [100/168], Loss: 0.7845
Epoch [45/150], Step [110/168], Loss: 0.6783
Epoch [45/150], Step [120/168], Loss: 0.6483
Epoch [45/150], Step [130/168], Loss: 2.3050
Epoch [45/150], Step [140/168], Loss: 2.0264
Epoch [45/150], Step [150/168], Loss: 1.9955
Epoch [45/150], Step [160/168], Loss: 1.9224
 Lr: 0.01 , Validataion accuracy is: 9.599477579451458 %
Epoch [46/150], Step [10/168], Loss: 0.9937
Epoch [46/150], Step [20/168], Loss: 0.9869
Epoch [46/150], Step [30/168], Loss: 0.9035
Epoch [46/150], Step [40/168], Loss: 0.8841
Epoch [46/150], Step [50/168], Loss: 0.7709
Epoch [46/150], Step [60/168], Loss: 0.8539
Epoch [46/150], Step [70/168], Loss: 0.7274
Epoch [46/150], Step [80/168], Loss: 0.6406
Epoch [46/150], Step [90/168], Loss: 0.6116
Epoch [46/150], Step [100/168], Loss: 0.7915
Epoch [46/150], Step [110/168], Loss: 0.6978
Epoch [46/150], Step [120/168], Loss: 0.6610
Epoch [46/150], Step [130/168], Loss: 2.2459
Epoch [46/150], Step [140/168], Loss: 2.1329
Epoch [

In [ ]:
train_function(k_resnet18, test_loader=f_full_loader, train=False, save_model_path=k_model_path_to_save)

In [120]:
train_function(k_resnet18, test_loader=val_loader, train=False)

Accuracy of the network on the 276 test images: 61.95652173913044 %


In [ ]:
# train_function(k_resnet18, test_loader=_loader, train=False)